# Fine-Tuning Mistral 7B for SQL assistant

Big LLMs cannot be used for generating SQL due to privacy reasons and also cost. Let's create a small LLM to help us

Total time required: < 1.5 hours.

This notebook shows you a simple example of how to LoRA finetune Mistral 7B. You need A100 with 40 GB memory. This is written using Google Collab but we should be able to use any other cloud.

Check out `mistral-finetune` Github repo to learn more: https://github.com/mistralai/mistral-finetune/

## Installation

Clone the `mistral-finetune` repo:


In [1]:
%cd /content/
!git clone https://github.com/mistralai/mistral-finetune.git

/content
Cloning into 'mistral-finetune'...
remote: Enumerating objects: 461, done.
remote: Counting objects: 100% (201/201), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 461 (delta 154), reused 136 (delta 112), pack-reused 260 (from 1)
Receiving objects: 100% (461/461), 247.79 KiB | 14.58 MiB/s, done.
Resolving deltas: 100% (238/238), done.


Install all required dependencies:

In [2]:
!pip install -r /content/mistral-finetune/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvtx_c

## Model download

In [3]:
!pip install huggingface_hub

In [4]:
# huggingface login , Enter your HF token below.
import os
os.environ["HF_TOKEN"] = "<Your HF Token>"

In [5]:
from huggingface_hub import snapshot_download
from pathlib import Path

mistral_models_path = Path.home().joinpath('mistral_models', '7B-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)

snapshot_download(repo_id="mistralai/Mistral-7B-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)

! cp -r /root/mistral_models/7B-v0.3 /content/mistral_models
! rm -r /root/mistral_models/7B-v0.3

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

consolidated.safetensors:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

params.json:   0%|          | 0.00/202 [00:00<?, ?B/s]

tokenizer.model.v3:   0%|          | 0.00/587k [00:00<?, ?B/s]

In [6]:
!ls /content/mistral_models

consolidated.safetensors  params.json  tokenizer.model.v3


## Prepare dataset

To ensure effective training, mistral-finetune has strict requirements for how the training data has to be formatted. Check out the required data formatting [here](https://github.com/mistralai/mistral-finetune/tree/main?tab=readme-ov-file#prepare-dataset).

In this example, let’s use the sql_create_context_v4.json dataset.This dataset has around 78k SQL records and text captions.

In [7]:
%cd /content/

/content


In [8]:
# make a new directory called data
!mkdir -p data

In [9]:
# download dataset to data folder
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="b-mc2/sql-create-context", filename="sql_create_context_v4.json", repo_type="dataset", local_dir="/content/data")

sql_create_context_v4.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

'/content/data/sql_create_context_v4.json'

## Split the data into train and test
sql_create_context_v4.json is a big json file with ~78k sql and captions. Lets split it into sql_test.jsonl and sql_train.jsonl. Also Mistral needs the input into certain jsonl format. JSONL is a text file where every line is a json record.
Format required for Mistral is like below.


In [10]:
{"messages": [{"role": "user", "content": "context: CREATE TABLE table_1973842_1 (type VARCHAR, location VARCHAR); question: What type is the school located in Macon, Georgia?"}, {"role": "assistant", "content": "SELECT type FROM table_1973842_1 WHERE location = \"Macon, Georgia\""}]}

{'messages': [{'role': 'user',
   'content': 'context: CREATE TABLE table_1973842_1 (type VARCHAR, location VARCHAR); question: What type is the school located in Macon, Georgia?'},
  {'role': 'assistant',
   'content': 'SELECT type FROM table_1973842_1 WHERE location = "Macon, Georgia"'}]}

### Function to split the list into train/test

In [11]:
seed = 42
import random
import json
def split_list(input_list, split_ratio=0.9, seed=seed):
    if seed is not None:
        random.seed(seed)

    shuffled_list = input_list.copy()
    random.shuffle(shuffled_list)
    split_index = int(len(shuffled_list) * split_ratio)
    train_set = shuffled_list[:split_index]
    test_set = shuffled_list[split_index:]

    return train_set, test_set

### Function to create a JSONL record

In [12]:
def write_prompt(que, file):
    q = que["question"]
    c = que["context"]
    a = que["answer"]
    line = {"messages": [{"role": "user", "content": f"context: {c}; question: {q}"}, {"role": "assistant", "content": a}]}
    file.write(json.dumps(line) + "\n")


### Using above functions, lets create files in data directory

In [13]:
from tqdm.notebook import tqdm
import json
input_file = "/content/data/sql_create_context_v4.json"
out_train_file = "/content/data/sql_train.jsonl"
out_test_file = "/content/data/sql_test.jsonl"

with open(input_file, "r") as jsonf:
    questions = json.load(jsonf)
    train_split, test_split = split_list([i for i  in range(len(questions))])
    with open(out_train_file, 'w') as file:
        for que in tqdm(train_split, desc="Processing Training Set"):
            write_prompt(questions[que], file)
    with open(out_test_file, 'w') as file:
        for que in tqdm(test_split, desc="Processing Test Set"):
            write_prompt(questions[que], file)

Processing Training Set:   0%|          | 0/70719 [00:00<?, ?it/s]

Processing Test Set:   0%|          | 0/7858 [00:00<?, ?it/s]

In [14]:
# navigate to this data directory
%cd /content/data
%ls

/content/data
sql_create_context_v4.json  sql_test.jsonl  sql_train.jsonl


In [15]:
# navigate to the mistral-finetune directory
%cd /content/mistral-finetune/

/content/mistral-finetune


In [16]:
# some of the training data doesn't have the right format,
# so we need to reformat the data into the correct format and skip the cases that don't have the right format:

!python -m utils.reformat_data /content/data/sql_train.jsonl

In [17]:
# eval data looks all good
!python -m utils.reformat_data /content/data/sql_test.jsonl

## Training
Below is the torchrun config yaml. I liked small yaml than hardcoding things in code. Torchrun also allows you to run it on multi-gpu environment.

In [18]:
# these info is needed for training
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [19]:
# define training configuration
# for your own use cases, you might want to change the data paths, model path, run_dir, and other hyperparameters

config = """
# data
data:
  instruct_data: "/content/data/sql_train.jsonl"  # Fill
  data: ""  # Optionally fill with pretraining data
  eval_instruct_data: "/content/data/sql_test.jsonl"  # Optionally fill

# model
model_id_or_path: "/content/mistral_models"  # Change to downloaded path
lora:
  rank: 64

seq_len: 8192
batch_size: 2
num_microbatches: 2
max_steps: 100
optim:
  lr: 1.e-4
  weight_decay: 0.1
  pct_start: 0.05

seed: 0
log_freq: 1
eval_freq: 100
no_eval: False
ckpt_freq: 100

save_adapters: True  # save only trained LoRA adapters. Set to `False` to merge LoRA adapter into the base model and save full fine-tuned model

run_dir: "/content/test_sql"  # Fill
"""

# save the same file locally into the example.yaml file
import yaml
with open('example.yaml', 'w') as file:
    yaml.dump(yaml.safe_load(config), file)


In [20]:
#If training Fails, you need to uncomment below remove this directory and start again.
#!rm -rf /content/test_sql

In [21]:
# train

!rm -rf /content/test_sql

!torchrun --nproc-per-node 1 -m train example.yaml

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
2024-08-21 23:02:10.899264: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-21 23:02:10.917347: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-21 23:02:10.939455: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-21 23:02:10.946057: E exte

## Evaluation

In [22]:
!pip install mistral_inference

In [23]:
from mistral_inference.transformer import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest


tokenizer = MistralTokenizer.from_file("/content/mistral_models/tokenizer.model.v3")
model = Transformer.from_folder("/content/mistral_models")
model.load_lora("/content/test_sql/checkpoints/checkpoint_000100/consolidated/lora.safetensors") # load lora

completion_request = ChatCompletionRequest(messages=[UserMessage(content="context: CREATE TABLE table_name_7 (socket VARCHAR, voltage VARCHAR, frequency VARCHAR, release_date VARCHAR); question: What is the socket related to the processor released on June 22, 2005, having a frequency of 1600MHz and voltage under 1.35V?")])

tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate([tokens], model, max_tokens=64, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

print(result)

SELECT socket FROM table_name_7 WHERE frequency = "1600mhz" AND release_date = "june 22, 2005" AND voltage < 1.35


In [24]:
!zip -r /content/test_sql.zip /content/test_sql

  adding: content/test_sql/ (stored 0%)
  adding: content/test_sql/metrics.eval.jsonl (deflated 21%)
  adding: content/test_sql/tb/ (stored 0%)
  adding: content/test_sql/tb/events.out.tfevents.1724281333.c3bdaa82a29f.1764.1.eval (deflated 26%)
  adding: content/test_sql/tb/events.out.tfevents.1724281333.c3bdaa82a29f.1764.0.train (deflated 73%)
  adding: content/test_sql/metrics.train.jsonl (deflated 77%)
  adding: content/test_sql/args.yaml (deflated 48%)
  adding: content/test_sql/checkpoints/ (stored 0%)
  adding: content/test_sql/checkpoints/checkpoint_000100/ (stored 0%)
  adding: content/test_sql/checkpoints/checkpoint_000100/consolidated/ (stored 0%)
  adding: content/test_sql/checkpoints/checkpoint_000100/consolidated/params.json (deflated 49%)
  adding: content/test_sql/checkpoints/checkpoint_000100/consolidated/lora.safetensors (deflated 21%)
  adding: content/test_sql/checkpoints/checkpoint_000100/consolidated/tokenizer.model.v3 (deflated 61%)


## Make sure you download the zip file before disconnecting the runtime.

## Next Steps:

We can use llamma.cpp to convert above model to gguf and run on smaller machine.
https://www.geeksforgeeks.org/how-to-convert-any-huggingface-model-to-gguf-file-format/